1. 데이터 가져오기
2. 데이터 전처리
    - 트레인 데이터세트와 테스트 데이터세트 준비 (과정 동일)
        - 순서 (column1: document=> 감상평, column2: label => 긍정(0) /부정(1) )
            - 1. Null 데이터 확인: DF.column1.isna().sum()
            - 2. Null 데이터 제거: DF.dropna(how='any', inplace=True) -> shape 확인 (DF.shape - 1.) 
            - 3. 중복 여부 확인: DF.column1.nunique() => 2.에서 중복된 값을 제외한 실제값이 들어있는 부분 출력
            - 4. 중복 데이터 제거: DF.drop_duplicates(subset=['column1'], inplace=True) -> shape 확인 (3.과 동일)
            - 5. 데이터 분포 확인(긍정 / 부정): DF.column2.value_counts()
3. 텍스트 전처리
    - 트레인 데이터세트와 테스트 데이터세트 전처리 (과정 동일)
        - 순서 
            - 1. 한글 이외의 문자 공백처리하고 strip: DF.column1 = DF.column1.str.replace('[ㄱ-ㅎㅏ-ㅣ가-힣]').str.strip() -> DF.head() 확인
                - 주의: 영어로 작성된 감상평이 있다면 정규표현식을 거치면서 공백처리됨: ''로 출력
            - 2. '' -> np.nan -> 제거: DF.column1.replace('', np.nan, inplace=True) -> DF.column1.isna().sum()의 결과로 날아가는 데이터의 개수를 확인할 수 있다. 
            - 3. Null 데이터 제거: DF.dropna(how='any', inplace=True) -> DF.shape로 확인
4. 데이터 저장 (train 데이터: DF1, test 데이터: DF2)
    - DF1.to_csv('폴더명/파일이름_전처리완료.csv', sep='\t', index=False)
    - DF2.to_csv('폴더명/파일이름_전처리완료.csv', sep='\t', index=False)
    - 데이터 생성되었는지 확인
5. 한글 처리 (SOYNLP 사용)
    - import joblib; scores = joblib.load('data/scores.pkl')
        - joblib으로 불러오는 scores는?
            - soynlp
                - 학습을 통해 단어 사전을 만듦: import urllib.requests; urllib.request.urlretrieve('링크/파일이름.txt', filename='폴더명/파일이름.txt')
                - 훈련 데이터를 문서로 분리: from soynlp import DoublespaceLineCorpus; corpus=DoublespaceLineCorpus('폴더명/파일이름.txt')
                - 말뭉치에서 단어 점수표 학습: from soynlp.word import WordExtractor; word_extractor = WordExtractor(); word_extractor.train(corpus); word_score_table=word_extractor.extract()
                - 토큰화: from soynlp.tokenizer import Ltokenizer; scores={word:score.cohesion_forward for word, score in word_score_table.items()}; l_tokenizer=LTokenizer(scores=scores); l_tokenizer.tokenize("예문", flatten=False)
                - joblib으로 저장: import joblib; joblib.dump(scores, '폴더명/scores.pkl') (2022-01-06, 21.Soynlp 참고)
                    - 명사 추출: from soynlp.noun import LRNounExtractor_v2; noun_extractor=NRNounExtractor_v2(verbose=True); nouns=noun_extractor.train_extract(corpus)
                        - joblib으로 저장: noun_scores={noun:scores[0] for noun, score in nouns.items() if len(noun) > 1}; joblib.dump(noun_scores, '폴더명/noun_scores.pkl')
    - 한글 vector화: 한글 vector화 -> X_train 데이터 (X_train: CountVectorize하기 위해 가공된 형태)
        - 1. 형태소로 tokenize: from soynlp.tokenizer import MAxScoreTokenizer; max_tokenizer=MaxScoreTokenizer(scores); text="예문"; word_list=max_tokenizer.tokenize(text)
        - 2. stop_text 설정: stop_text='직접 설정한 불용어 쉼표 없이 나열'; stop_list=stop_text.split()
        - 3. 불용어를 제거한 형태소 추출: word_list = [word for word in word_list if word not in stop_list]
        - 4. 다시 string으로 만들기: review=' '.join(word_list)
    - vector화한 문자열을 X_train, X_test, y_train, y_test에 적용하기 
        - X_train에 적용: X_train=[]; for document in DF1.column1[:3]: ; word_list=max_tokenizer.tokenize(column1); word_list=[word for word in word_list if word not in stop_list]; tokenized_str=' '.join(word_list); X_train.append(tokenized_str)
            - X_train 확인
            - DF1.column1[:3] : 원래 모양과 비교
        - X_test 적용: X_train과 동일
        - y_train: y_train=DF1.column2.values
        - y_test: y_test=DF2.column2.values
6. Feature 변환: CounterVectorizer
    - from sklearn.feature_extraction.text import CountVectorizer; cvect=CountVectorizer(); cvect.fit(X_train); X_train_cv=cvect.transform(X_train); X_test_cv=cvect.transform(X_test) -> X_train_cv.shape, X_test.shape 확인
7. 모델 생성 / 학습 / 평가
    - 모델 import -> 객체화 -> 객체화된 모델.fit(Feature변환한 X_train, y_train); 객체화된 모델.score(Feature변환한 X_test, y_test)
8. 실제 테스트 순서
    - 1. review1, review2 문자열로 들고오기
    - 2. 전처리
        - 주의: DF는 정규표현식을 사용할 수 있지만, 문자열은 정규표현식을 사용할 수 없다
            - 해결: import re; review1=re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', review1).strip(); review2=re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', review2).strip()
    - 3. 형태소 분석 및 불용어 처리 (review1, review2 과정 동일)
        - words=max_tokenizer.tokenize(review1); words=[word for word in words if word not in stop_list]; review1=' '.join(words)
    - 4. Feature 변환 (CountVectorizer)
        - review=[review1, review2]; review_cv=cvect.transform(review)
    - 5. 예측
        - 모델.predict(review_cv)
9. 튜닝 (모델의 성능향상을 위해 파라미터 조절) : GridSearchCV 사용 => 최적의 파라미터 찾고, 교차검증을 해주는 역할을 한다.
    - from sklearn.model_selection import GridSearchCV; from sklearn.pipeline import Pipeline
    - pipeline 설정: pipeline=PipeLine([('cvect', CountVectorizer()), ('lrc', LogisticRegression(random_state=2022))]) # (객체화된 클래스명, 모델 클래스)
    - 파라미터 설정: params={'cvect__ngram_range': [(1,1),(1,2)], 'cvect__max_df':[0.9, 0.99], 'cvect__min_df': [1,3], 'lrc__C':[1,5]}
    - grid_pipe=GridSearchCV(pipeline, param_grid=params, scoring='accuracy', cv=3)
    - 최적의 파라미터 확인: grid_pipe.best_params_
    - 최적의 측정기: grid_pipe.best_estimator_.score(X_test, y_test)
10. 다른 모델을 사용하여 정확도 측정하고 비교해보기